# EDA

## Exploratory Data Analysis

En este archivo nos centraremos en analizar nuestros datos, revisar los datasets proporcionados y tratar de decifrar que contienen y que acciones tomar para poder mejorar nuestros datos.

In [1]:
#Importamos las librerias con las que vamos a trabajar solamente
import pandas as pd
import ast

Para comenzar podemos observar que nuestros datos se encuentra en un archivo formato
.json tipo diccionario.

In [2]:
#Importamos con pandas nuestros archivos json
df_steamGames = pd.read_json(r'..\DataRaw\\output_steam_games.json', lines= True)

print(f'La información de nuestro dataframe de SteamGames:\n')
df_steamGames.info()


La información de nuestro dataframe de SteamGames:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Cada columna tiene más de 120k entradas y de la información de nuestro df, se puede observar
que aproximadamente solo el 25% de nuestro df tiene datos, unas columnas tienen más datos que otras
pero rondan casi por los mismos valores. Aquí enfrentamos un problema grande, si se opta por eliminar
todos los datos vacios perderiamos mucha información que en caso de querer hacer un estudio
muy profundo realmente nos veríamos muy afectado. Sin embargo, en este proyecto no es necesario
tener una información tan detallada o no es de gran relevancia, por eso vamos a trabajar solo con 
las filas que contengan los datos completos o la mayoría de los mismo.

Entonces en nuestra ETL del df de juegos steam y los demas datsets vamos a eliminar los valores vacios.

In [3]:
#Ahora vamos a revisar si existen valores repetidos
repets = df_steamGames['publisher'].unique()
print(len(repets))


8240


Podemos observar que solo para publisher de tener más de 24k datos, al final solo
tenemos 8k diferentes, por lo tanto sabemos que hay publishers repetidos, sin embargo eso es posible
ya que un publisher puede tener varios juegos diferentes.


Ahora empezamos a trabajar con los otros dos datasets.

Para el data set de australian items tenemos que utilizar otro estilo de decodificación, ya que
de otra manera tendremos errores leyendo el archivo con pandas.

In [4]:
#Leemos el archivo linea por linea para poder decodificar
rows = []

with open(r'..\DataRaw\\australian_users_items.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

f.close()
    
df_australian_items = pd.DataFrame(rows)
df_australian_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Como podemos observar en la última columna tenemos información anidada, así que tenemos que en la ETL
vamos a tener que desanidar esa columna para poder generar un dataframe con toda la información
por columnas separadas. Tambien podemos ver el tiempo de ejecución, en este caso fue bastante, esto nos
indica que el dataset es muy pesado y tenemos que optimizar este formato de lectura en la ETL.

In [5]:
#Analizando la cantidad de datos vacios en este ultimo dataframe
df_australian_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Para este dataset podemos ver que tenemos 88310 entradas. En este dataframe tenemos
todos los datos completos, por lo cual en la ETL solo va a ser necesario desanidar
la información de la columna de items para poder expandir este dataframe


Ahora procedemos con el tercer dataset, el user reviews.

In [6]:
#Tenemos el mismo problema que con el dataset pasado, así que utilizaremos la misma decodificación
rows = []

with open(r'..\DataRaw\\australian_user_reviews.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

f.close()
    
df_aust_reviews = pd.DataFrame(rows)
df_aust_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Para este data set podemos observar que el tiempo de procesamiento fue mucho menor, sin embargo,
tenemos otra columna con información anidada, así que vamos a tener que desanidar también para la ETL
de este data set.

Ahora, viendo los valores vacios...

In [7]:
df_aust_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Podemos observar que en este dataset contamos con la información completa, de las 25799 entradas
tenemos el mismo numero de entradas NO nulas, por lo tanto no va a ser necesario eliminar datos vacios
lo más importante va a ser el desanidar la columna "reviews", que será de donde encontremos nuestras
recomendaciones con el MLP.

### **--- RECAPITULANDO ---**

1._ Para el dataset de steam games es necesario eliminar todos los datos vaciós y podemos ordenar
los alfabeticamente los publishers para mayor orden

2._ Para el australian items es necesario desanidar la columna de items y así expandir nuestro dataframe

3._ Para el australian reviews es necesario hacer lo mismo, desanidar la columna de reviews y así expandir
el dataframe para tener más datos.
